In [1]:
# from argoverse.map_representation.map_api import ArgoverseMap
# from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
# from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader
# from argoverse.visualization.visualize_sequences import viz_sequence

# from statistics import mean
# import glob
import torch
from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils
# import torch.nn as nn

from sgan.data.data import Argoverse_Social_Data, collate_traj_social

import numpy as np

In [2]:
argoverse_train = Argoverse_Social_Data('../../deep_prediction/data/train/data/')
argoverse_val   = Argoverse_Social_Data('../../deep_prediction/data/val/data')
argoverse_test  = Argoverse_Social_Data('../../deep_prediction/data/test_obs/data')

batch_size = 4

train_loader = DataLoader(argoverse_train, batch_size=batch_size,
                    shuffle=True,collate_fn=collate_traj_social)
val_loader = DataLoader(argoverse_val, batch_size=batch_size,
                    shuffle=True,collate_fn=collate_traj_social)
test_loader = DataLoader(argoverse_test, batch_size=batch_size,
                    shuffle=True,collate_fn=collate_traj_social)

In [3]:
dataiter = iter(train_loader)
batch = dataiter.next()

train_agent = batch['train_agent']
gt_agent = batch['gt_agent']
neighbour = batch['neighbour']
neighbour_gt = batch['neighbour_gt']

obs_traj = train_agent[0].unsqueeze(0)
obs_traj = torch.cat((obs_traj, neighbour[0]),0)

pred_traj_gt = gt_agent[0].unsqueeze(0)
pred_traj_gt = torch.cat((pred_traj_gt, neighbour_gt[0]),0)

ped_count = obs_traj.shape[0]
seq_start_end = [[0, ped_count]] # last number excluded

print(len(neighbour))

for i in range(1, len(neighbour)):
    obs_traj = torch.cat((obs_traj, train_agent[i].unsqueeze(0)), 0)
    obs_traj = torch.cat((obs_traj, neighbour[i]), 0)
    
    pred_traj_gt = torch.cat((pred_traj_gt, gt_agent[i].unsqueeze(0)), 0)
    pred_traj_gt = torch.cat((pred_traj_gt, neighbour_gt[i]), 0)
          
    seq_start_end.append([ped_count, obs_traj.shape[0]])
    ped_count = obs_traj.shape[0]

obs_traj_rel = torch.zeros(obs_traj.shape)
obs_traj_rel[:,1:,:] = obs_traj[:,1:,:] -  obs_traj[:,:-1,:]    

pred_traj_gt_rel = torch.zeros(pred_traj_gt.shape)
pred_traj_gt_rel[:,1:,:] = pred_traj_gt[:,1:,:] - pred_traj_gt[:,0:-1,:]

seq_start_end = torch.tensor(seq_start_end)
## 
obs_traj = obs_traj.transpose_(0,1) 
obs_traj_rel = obs_traj_rel.transpose_(0,1) 
pred_traj_gt = pred_traj_gt.transpose_(0,1) 
pred_traj_gt_rel = pred_traj_gt_rel.transpose_(0,1) 



4


In [4]:
print(seq_start_end)
print(obs_traj.shape)
print(obs_traj_rel.shape)
print(pred_traj_gt.shape)
print(pred_traj_gt_rel.shape)
print(non_linear_ped_linear_ped_linear_ped_linear_ped.shape)

tensor([[ 0,  2],
        [ 2, 19],
        [19, 21],
        [21, 27]])
torch.Size([20, 27, 2])
torch.Size([20, 27, 2])
torch.Size([30, 27, 2])
torch.Size([30, 27, 2])


NameError: name 'non_linear_ped_linear_ped_linear_ped_linear_ped' is not defined

In [ ]:
pred_traj = torch.rand(pred_traj_gt.shape)

loss_mask = torch.ones(len(neighbour), 50)
loss_mask = loss_mask[:, 20:50]

print(loss_mask.unsqueeze(dim=0).transpose_(1,2).shape)
print(pred_traj_gt.permute(1, 0, 2).shape)
print(pred_traj.permute(1, 0, 2).shape)

loss = (loss_mask.unsqueeze(dim=0).transpose_(1,2) *
            (pred_traj_gt.permute(1, 0, 2) - pred_traj.permute(1, 0, 2))**2)

print(loss.shape)

In [ ]:
def poly_fit(traj, traj_len=30, threshold=0.002):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[:, 0], 2, full=True)[1]
    res_y = np.polyfit(t, traj[:, 1], 2, full=True)[1]
    
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

In [ ]:
dataiter = iter(train_loader)
batch = dataiter.next()

train_agent = batch['train_agent']
gt_agent = batch['gt_agent']
neighbour = batch['neighbour']
neighbour_gt = batch['neighbour_gt']

obs_traj = train_agent[0].unsqueeze(0)
obs_traj = torch.cat((obs_traj, neighbour[0]),0)

pred_traj_gt = gt_agent[0].unsqueeze(0)
pred_traj_gt = torch.cat((pred_traj_gt, neighbour_gt[0]),0)

ped_count = obs_traj.shape[0]
seq_start_end = [[0, ped_count]] # last number excluded

non_linear_ped = []
_non_linear_ped = [poly_fit(np.array(gt_agent[0]))]

for j in range(0,ped_count-1):
    _non_linear_ped.append(poly_fit(np.array(neighbour_gt[0][j])))
non_linear_ped += _non_linear_ped

print(obs_traj.shape)
print(len(non_linear_ped))

for i in range(1, len(neighbour)):
    obs_traj = torch.cat((obs_traj, train_agent[i].unsqueeze(0)), 0)
    obs_traj = torch.cat((obs_traj, neighbour[i]), 0)

    pred_traj_gt = torch.cat((pred_traj_gt, gt_agent[i].unsqueeze(0)), 0)
    pred_traj_gt = torch.cat((pred_traj_gt, neighbour_gt[i]), 0)

    seq_start_end.append([ped_count, obs_traj.shape[0]])

    num_peds_considered = obs_traj.shape[0] - ped_count
    ped_count = obs_traj.shape[0]

    _non_linear_ped = [poly_fit(np.array(gt_agent[i]))]

    for j in range(0, num_peds_considered-1):
        _non_linear_ped.append(poly_fit(np.array(neighbour_gt[i][j])))

    non_linear_ped += _non_linear_ped

obs_traj_rel = torch.zeros(obs_traj.shape)
obs_traj_rel[:,1:,:] = obs_traj[:,1:,:] -  obs_traj[:,:-1,:]    

pred_traj_gt_rel = torch.zeros(pred_traj_gt.shape)
pred_traj_gt_rel[:,1:,:] = pred_traj_gt[:,1:,:] - pred_traj_gt[:,0:-1,:]

seq_start_end = torch.tensor(seq_start_end)
non_linear_ped = torch.tensor(non_linear_ped)
## 
obs_traj = obs_traj.transpose_(0,1) 
obs_traj_rel = obs_traj_rel.transpose_(0,1) 
pred_traj_gt = pred_traj_gt.transpose_(0,1) 
pred_traj_gt_rel = pred_traj_gt_rel.transpose_(0,1) 

In [ ]:
print(obs_traj_rel.shape)
print(non_linear_ped.shape)

In [ ]:
from datetime import datetime
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d-%m-%Y::%H:%M:%S")
print("date and time =", dt_string)	

In [ ]:
import os
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y::%H:%M:%S")

dataset_name = 'argoverse'
model_name = 'sgan'

wr_dir = '../runs/' + dataset_name + '/' + model_name + '/' + dt_string + '/'
os.makedirs(wr_dir, exist_ok=True)

In [ ]:
import argparse
import gc
import logging
import os
import sys
import time

from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim

from sgan.data.loader import data_loader
from sgan.losses_argo import gan_g_loss, gan_d_loss, l2_loss
from sgan.losses_argo import displacement_error, final_displacement_error

from sgan.models_argo import TrajectoryGenerator, TrajectoryDiscriminator
from sgan.utils import int_tuple, bool_flag, get_total_norm
from sgan.utils import relative_to_abs, get_dset_path

from sgan.data.data import Argoverse_Social_Data, collate_traj_social
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import numpy as np
torch.backends.cudnn.benchmark = True


In [ ]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight)
        
def get_dtypes(use_gpu=True):
    long_dtype = torch.LongTensor
    float_dtype = torch.FloatTensor
    if use_gpu == 1:
        long_dtype = torch.cuda.LongTensor
        float_dtype = torch.cuda.FloatTensor
    return long_dtype, float_dtype        

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
long_dtype, float_dtype = get_dtypes()

argoverse_val   = Argoverse_Social_Data('../../deep_prediction/data/val/data')
argoverse_test  = Argoverse_Social_Data('../../deep_prediction/data/test_obs/data')

In [ ]:
batch_size = 1
val_loader = DataLoader(argoverse_val, batch_size=batch_size,
                    shuffle=True, num_workers=2,collate_fn=collate_traj_social)
test_loader = DataLoader(argoverse_test, batch_size=batch_size,
                shuffle=True, num_workers=2,collate_fn=collate_traj_social)

In [ ]:
generator = TrajectoryGenerator(
    obs_len=20,
    pred_len=30,
    embedding_dim=16,
    encoder_h_dim=32,
    decoder_h_dim=32,
    mlp_dim=64,
    num_layers=1,
    noise_dim=(8, ),
    noise_type='gaussian',
    noise_mix_type='global',
    pooling_type='pool_net',
    pool_every_timestep=0,
    dropout=0,
    bottleneck_dim=32,
    neighborhood_size=2.0,
    grid_size=8,
    batch_norm=False)

In [ ]:
generator.apply(init_weights)
generator.type(float_dtype).train()

In [ ]:
output_dir = os.getcwd()
checkpoint_name = 'gan_test'
restore_path = os.path.join(output_dir,'%s_with_model.pt' % checkpoint_name)

In [ ]:
restore_path

In [ ]:
checkpoint = torch.load(restore_path)
generator.load_state_dict(checkpoint['g_state'])

In [ ]:
def poly_fit(traj, traj_len=30, threshold=0.002):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[:, 0], 2, full=True)[1]
    res_y = np.polyfit(t, traj[:, 1], 2, full=True)[1]
    
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0

In [ ]:
dataiter = iter(val_loader)
batch = dataiter.next()

train_agent = batch['train_agent']
gt_agent = batch['gt_agent']
neighbour = batch['neighbour']
neighbour_gt = batch['neighbour_gt']

obs_traj = train_agent[0].unsqueeze(0)
obs_traj = torch.cat((obs_traj, neighbour[0]),0)

pred_traj_gt = gt_agent[0].unsqueeze(0)
pred_traj_gt = torch.cat((pred_traj_gt, neighbour_gt[0]),0)

ped_count = obs_traj.shape[0]
seq_start_end = [[0, ped_count]] # last number excluded

non_linear_ped = []
_non_linear_ped = [poly_fit(np.array(gt_agent[0]))]

for j in range(ped_count-1):
    _non_linear_ped.append(poly_fit(np.array(neighbour_gt[0][j])))
non_linear_ped += _non_linear_ped

for i in range(1, len(neighbour)):
    obs_traj = torch.cat((obs_traj, train_agent[i].unsqueeze(0)), 0)
    obs_traj = torch.cat((obs_traj, neighbour[i]), 0)

    pred_traj_gt = torch.cat((pred_traj_gt, gt_agent[i].unsqueeze(0)), 0)
    pred_traj_gt = torch.cat((pred_traj_gt, neighbour_gt[i]), 0)

    seq_start_end.append([ped_count, obs_traj.shape[0]])

    num_peds_considered = obs_traj.shape[0] - ped_count
    ped_count = obs_traj.shape[0]

    _non_linear_ped = [poly_fit(np.array(gt_agent[i]))]

    for j in range(num_peds_considered-1):
        _non_linear_ped.append(poly_fit(np.array(neighbour_gt[i][j])))

    non_linear_ped += _non_linear_ped

obs_traj_rel = torch.zeros(obs_traj.shape)
obs_traj_rel[:,1:,:] = obs_traj[:,1:,:] -  obs_traj[:,:-1,:]    

pred_traj_gt_rel = torch.zeros(pred_traj_gt.shape)
pred_traj_gt_rel[:,1:,:] = pred_traj_gt[:,1:,:] - pred_traj_gt[:,0:-1,:]

seq_start_end = torch.tensor(seq_start_end)
non_linear_ped = torch.tensor(non_linear_ped).cuda()

## 
obs_traj = obs_traj.transpose_(0,1).cuda() 
obs_traj_rel = obs_traj_rel.transpose_(0,1).cuda() 
pred_traj_gt = pred_traj_gt.transpose_(0,1).cuda() 
pred_traj_gt_rel = pred_traj_gt_rel.transpose_(0,1).cuda() 

################################################################

linear_ped = 1 - non_linear_ped

loss_mask = torch.ones(pred_traj_gt.shape[1],30).cuda()
#             loss_mask = loss_mask[:, args.obs_len:].cuda()

pred_traj_fake_rel = generator(obs_traj, obs_traj_rel, seq_start_end)
pred_traj_fake = relative_to_abs(pred_traj_fake_rel, obs_traj[-1])

In [ ]:
pred_traj_fake.shape

In [ ]:
from matplotlib import pyplot as plt
obs_traj_ = np.array(obs_traj.cpu())
pred_traj_gt_ = np.array(pred_traj_gt.cpu())
pred_traj_fake_ = pred_traj_fake.cpu().detach().numpy()

j = 27
i = seq_start_end[j][0]

print(seq_start_end[j])
plt.plot(obs_traj_[:,i,0],obs_traj_[:,i,1])
plt.plot(pred_traj_gt_[:,i,0],pred_traj_gt_[:,i,1], 'r')
plt.plot(pred_traj_fake_[:,i,0],pred_traj_fake_[:,i,1], 'b')

for ped in range(i+1, seq_start_end[j][1]):
    plt.plot(obs_traj_[:,ped,0],obs_traj_[:,ped,1], 'g')
    print(ped)

plt.show()

In [ ]:
import pickle
import pandas as pd
dogs_dict = { 'Ozzy': 3, 'Filou': 8, 'Luna': 5, 'Skippy': 10, 'Barco': 12, 'Balou': 9, 'Laika': 16 }
filename = 'dogs'
outfile = open(filename,'wb')
pickle.dump(dogs_dict,outfile)
outfile.close()

filename = '../../deep_prediction/forecasting_sample/data/2645.csv'
a = pd.read_csv(filename)
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [226]:
a

,TIMESTAMP,TRACK_ID,OBJECT_TYPE,X,Y,CITY_NAME
0,3.159682e+08,00000000-0000-0000-0000-000000000000,AV,419.354578,1125.928065,MIA
1,3.159682e+08,00000000-0000-0000-0000-000000023470,OTHERS,404.729217,1253.006591,MIA
2,3.159682e+08,00000000-0000-0000-0000-000000023463,OTHERS,491.967704,1147.286581,MIA
3,3.159682e+08,00000000-0000-0000-0000-000000023476,OTHERS,473.827482,1146.672473,MIA
4,3.159682e+08,00000000-0000-0000-0000-000000023478,OTHERS,419.641337,1252.034538,MIA
...,...,...,...,...,...,...
752,3.159682e+08,00000000-0000-0000-0000-000000023475,OTHERS,427.751095,1067.468674,MIA
753,3.159682e+08,00000000-0000-0000-0000-000000023464,OTHERS,499.183176,1147.719295,MIA
754,3.159682e+08,00000000-0000-0000-0000-000000023494,OTHERS,419.411560,1116.500379,MIA
755,3.159682e+08,00000000-0000-0000-0000-000000023465,AGENT,417.073771,1161.073060,MIA


In [227]:
batch

{'train_agent': tensor([[[  0.0000,   0.0000],
          [ -0.9568,  -0.6659],
          [ -1.0622,  -0.7724],
          [ -2.3832,  -1.3139],
          [ -2.9024,  -1.4538],
          [ -3.2851,  -1.6489],
          [ -3.9953,  -2.0400],
          [ -5.0242,  -2.5567],
          [ -6.0222,  -2.8153],
          [ -6.1710,  -3.0095],
          [ -6.8950,  -3.4038],
          [ -8.0496,  -3.8297],
          [ -8.3546,  -4.0119],
          [ -8.9927,  -4.3150],
          [ -9.7466,  -4.7698],
          [-10.6107,  -5.1745],
          [-11.1077,  -5.4027],
          [-11.6356,  -5.6757],
          [-12.3293,  -5.9826],
          [-12.9784,  -6.2139]]]), 'gt_agent': tensor([[[-13.4891,  -6.5254],
          [-14.0291,  -6.7324],
          [-14.6798,  -7.0661],
          [-15.2654,  -7.3750],
          [-16.2067,  -7.8261],
          [-16.2067,  -7.8261],
          [-17.2483,  -8.2575],
          [-17.7901,  -8.3860],
          [-18.6913,  -8.8063],
          [-18.5116,  -8.8917],
          [

In [ ]:
from argoverse.map_representation.map_api import ArgoverseMap
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from argoverse.data_loading.argoverse_forecasting_loader import ArgoverseForecastingLoader

avm = ArgoverseMap()

argoverse_forecasting_loader = ArgoverseForecastingLoader('argoverse-forecasting/') #simply change to your local path of the data